In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_curve,roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("data.csv")
df = data.copy()
df.drop(["id","Unnamed: 32"],axis=1,inplace=True)

#Kötü huylu olmasının üzerine yoğunlaştığımız için 'M' gördüğümüz yere 1 değerini vereceğiz

df.diagnosis = [1 if kod == "M" else 0 for kod in df.diagnosis]


y = df["diagnosis"]
X = df.drop("diagnosis",axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

model = LogisticRegression(random_state=0)
model.fit(X_train,y_train)
tahmin = model.predict(X_test)

#Hata matrisi yapısını oluşturma işlemleri

cm = confusion_matrix(y_test,tahmin)
print(cm)#70 = TP | 1 = FN | 2 = FP | 41 = TN

acs = accuracy_score(y_test,tahmin)
print(acs) #0.9736842105263158 oranında doğruluk oranı vardır yani TP İLE TN'Yİ toplayıp tüm toplama bölme işlemi
#1-Doğruluk (Accuracy): (TP + TN) / (TP + FP + TN + FN) | Modeli yüzde kaç tahmin ettiğini verir
cr = classification_report(y_test,tahmin)
print(cr) 
#3- Hassasiyet(Kesinlik)(Precision): TP / (TP + FP) | Hassasiyet, modelinizin spam olarak tahmin ettiği
#değerlerin ne kadarının gerçekten spam olduğunu ölçer. Yani, modelinizin spam olarak etiketlediği örneklerin kaçı 
#gerçekten spamdır. Bu, modelinizin yanlış pozitif (spam olmayan bir e-postayı spam olarak işaretleme) hatalarını ne
#kadar az yaptığınızı gösterir

#4- Duyarlılık (Recall): TP / (TP + FN) | Duyarlılık, gerçek spam olan örneklerin kaçını doğru bir şekilde 
#tespit ettiğinizi ölçer

#5- F1-Score (F1-Skoru): 2 * (Hassasiyet * Duyarlılık) / (Hassasiyet + Duyarlılık) |
#Hem kesinlik hem de duyarlılık üzerinden bir işlem gerçekleştiriyor | 

In [ ]:
#ROC Curve (ROC Eğrisi) = TP ile FP oranları birbiriyle kıyaslamasını gösteren bir eğri yapısı.
auc = roc_auc_score(y_test,tahmin)
#fpr = false oran yapısı
#tpr = doğruluk oran yapısı
#thresold = allows us to map the sigmoid output of a binary classification to a binary category.
fpr,tpr,thresold = roc_curve(y_test,model.predict_proba(X_test)[:,1])
plt.plot(fpr,tpr,label="Model AUC (Alan=%0.2f)" % auc)
plt.plot([0,1],[0,1],"r--")
plt.xlabel("False Positive Oranı")
plt.ylabel("True Positive Oranı")
plt.title("ROC")
plt.legend(loc="lower right")
plt.show()